# XL-BOMD
## see https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import XL_BOMD

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/4_XL-BOMD'}
xl_bomd_params={'k':6}

md =  XL_BOMD(xl_bomd_params=xl_bomd_params,
              seqm_parameters=seqm_parameters, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule, Temperature = 400)
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 20, remove_com=[True, 1], Info_log=True)

Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   504.34   2.607617e-01 -1.360431e+00 -1.099670e+00 ||    381.70   1.973523e-01 1.419623e+00 1.616976e+00 || 
     2   475.22   2.457086e-01 -1.346130e+00 -1.100422e+00 ||    425.18   2.198373e-01 1.396738e+00 1.616576e+00 || 
     3   426.59   2.205630e-01 -1.321501e+00 -1.100938e+00 ||    480.25   2.483076e-01 1.367714e+00 1.616022e+00 || 
     4   366.13   1.893041e-01 -1.289250e+00 -1.099946e+00 ||    537.12   2.777116e-01 1.337895e+00 1.615607e+00 || 
     5   301.16   1.557102e-01 -1.254642e+00 -1.098932e+00 ||    584.59   3.022588e-01 1.313067e+00 1.615326e+00 || 
     6   238.56   1.233434e-01 -1.221916e+00 -1.098573e+00 ||    612.14   3.164983e-01 1.298646e+00 1.615145e+00 || 
     7   184.75   9.552514e-02 -1.193993e+00 -1.098468e+00 ||    612.44   3.166569e-01 1.298411e+00 1.615068e+00 || 
     8   144.65   7.478964e-02 -1.173436e+00 -1.098646e+00 ||    583.81   3.018548e-01 1.313250e+00 1.615

## Damping (Langevin)

In [3]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/4_XL-BOMD_damp'}
xl_bomd_params={'k':6}

md =  XL_BOMD(xl_bomd_params=xl_bomd_params, damp=100.0,
              seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule, Temperature = 400)
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   503.18   2.601635e-01 -1.360614e+00 -1.100450e+00 ||    385.34   1.992351e-01 1.420770e+00 1.620005e+00 || 
     2   486.94   2.517666e-01 -1.345797e+00 -1.094031e+00 ||    428.74   2.216735e-01 1.401220e+00 1.622894e+00 || 
     3   438.08   2.265071e-01 -1.320381e+00 -1.093874e+00 ||    474.05   2.451010e-01 1.373095e+00 1.618196e+00 || 
     4   365.93   1.892025e-01 -1.285074e+00 -1.095871e+00 ||    512.14   2.647955e-01 1.345560e+00 1.610355e+00 || 
     5   289.50   1.496840e-01 -1.249187e+00 -1.099503e+00 ||    536.42   2.773503e-01 1.320193e+00 1.597543e+00 || 
     6   219.26   1.133658e-01 -1.214387e+00 -1.101022e+00 ||    540.18   2.792936e-01 1.301993e+00 1.581286e+00 || 
     7   162.50   8.401837e-02 -1.185579e+00 -1.101561e+00 ||    540.03   2.792180e-01 1.294337e+00 1.573555e+00 || 
     8   139.94   7.235205e-02 -1.166973e+00 -1.094621e+00 ||    533.28   2.757260e-01 1.299902e+00 1.575